# Toegepaste Machine Learning

## Practical Assignment 3

### Important Notes:
1. Submit through **Canvas** in electronic form before 11:59pm on Tuesday, May 31, 2022
2. No late homework will be accepted.
3. This is a group-of-two assignment
4. The submitted file should be in ipynb format
5. The assignment is worth it 10 points
6. **Please make sure you are in a group on canvas!**
7. Question can be asked on the discussion tab on canvas.
8. The indication **optional** means that the question is optional; you won't lose any points if you do not do that part of the assignment, nor will you gain if you do it.

### Software:
We will be using Python programming language throughout this course. Further we will be using:
+ IPython Notebooks (as an environment)
+ Numpy
+ Pandas
+ Scikit-learn


### Background:

This practical assignment will be covering clustering and working with text. 

For the assignment, please download the dataset on [Movies](https://drive.google.com/drive/folders/1u_cDY-ZIJQKZSdgadzsbI0Q_dj4Jhy41?usp=sharing), or simply execute the command below. 

The folder contains a number of files, of which you will be using two: plot_summaries.txt and movie.metadata.tsv. Both are tab separated files. The former, i.e. plot_summaries.txt, contains the plot summaries of 42,306 movies extracted from the November 2, 2012 dump of English-language Wikipedia. Each line contains the Wikipedia movie ID (which indexes into movie.metadata.tsv) followed by the summary. The latter file, i.e. movie.metadata.tsv contains metadata for 81,741 movies, extracted from the Noverber 4, 2012 dump of Freebase. Freebase is a knowledge base (similar to a database) that contains information about different Entities (including movies). The file is tab-separated with the following columns:

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie name
4. Movie release date
5. Movie box office revenue
6. Movie runtime
7. Movie languages (Freebase ID:name tuples)
8. Movie countries (Freebase ID:name tuples)
9. Movie genres (Freebase ID:name tuples)

The goal of this assignment will be to cluster movies.

**Important Note**: This third assignment is not as instructive as the first assignment. The first assignment guided you step-by-step through all the preprocessing, training-validation-testing setup, etc. This assignment does not do so, but it leaves it up to you to decide how to use the data and design your experiments.

In [4]:
! mkdir MovieSummaries

! gdown "https://drive.google.com/uc?id=1tyun9bJq63hu9A9p11B75HjMQTCG5JpP" -O ./MovieSummaries/movie.metadata.tsv
! gdown "https://drive.google.com/uc?id=1Aeo9WwaKX39Z2zV374zLXW-ndlzcWYDn" -O ./MovieSummaries/moviesrep.csv
! gdown "https://drive.google.com/uc?id=1AnELzvfDWc6NAyV55lfrcVpAoIWzZAAl" -O ./MovieSummaries/plot_summaries.txt
! gdown "https://drive.google.com/uc?id=1CxsqcbtYzAR8S3zLSsstgC6QQa1mUPqd" -O ./MovieSummaries/README.txt

zsh:1: command not found: gdown
zsh:1: command not found: gdown
zsh:1: command not found: gdown
zsh:1: command not found: gdown


### Part 1: Import the data

We import both files and performing a join (merging the two files) using the Wikipedia ID (WID) to match the movies that appear in summaries to those that appear in the metadata. If a movie does not appear in either file, it is not included in the final list.

In [9]:
import pandas as pd
metadata = pd.read_csv('movie.metadata.tsv',sep="\t", header = None,
                        names=['WID', 'FID', 'Name', 'Release', 'Revenue', 
                               'Runtime', 'Languages', 'Countries', 'Genres'])
summaries = pd.read_csv('plot_summaries.txt',sep="\t", header = None,
                         names=['WID', 'Text'])
films = pd.merge(metadata, summaries, on='WID')

A movie may have more than one genre. We extract the first genre that characterizes the movie. Some movies may not have any corresponing genres.

In [10]:
import ast

genres = []
for film in films.values:
    exist = False
    g = ast.literal_eval(film[8])
    # Get the first genre for this movie
    for key in g:
        exist = True
        genres.append(g[key])
        break
    # If there is no genre for this movie
    if exist is False:
        genres.append('')

Consider only movies in four genres: 'Drama', 'Comedy', 'Science Fiction', 'Action'. Then sort them by Revenue they had in the cinemas, and get the top 150 most popular ones.

In [11]:
# Merge the films with the genre into a single Dataframe
genres = pd.Series(genres, name='Genre')
films_genre = pd.concat([films, genres], axis=1)

# Get only movies about the four following genres
films_genre_ind = films_genre.set_index('Genre')
movie_genres = ['Drama', 'Comedy', 'Science Fiction', 'Action']
genre150 = pd.DataFrame()
for mg in movie_genres:
    genre150 = genre150.append(films_genre_ind.loc[mg])

# Get the top-150 of those
top150 = (genre150.sort_values(by='Revenue',ascending=False)[0:150]).reset_index()[['Name','Text','Genre']]


What you are left with are 150 movies.

In [12]:
# Show the dataframe of the top150 dataset you will be working on.
top150

,Name,Text,Genre
0,The Avengers,"The Asgardian Loki encounters the Other, the ...",Science Fiction
1,Harry Potter and the Deathly Hallows – Part 2,{{further2}} {{See also}} As Lord Voldemort r...,Drama
2,Star Wars Episode I: The Phantom Menace,The Galactic Republic is in a period of decli...,Science Fiction
3,Star Wars Episode III: Revenge of the Sith,"As the Clone Wars rage on, the Galactic Repub...",Science Fiction
4,E.T. the Extra-Terrestrial,The film opens in a California forest as a gro...,Science Fiction
...,...,...,...
145,Brothers,Sam Cahill and Tommy Cahill are brothers. A ...,Drama
146,Regarding Henry,"Ambitious, callous, narcissistic, and at times...",Drama
147,Time Bandits,Eleven-year-old Kevin has a vivid imagination ...,Science Fiction
148,Naruto the Movie: Snow Princess' Book of Ninja...,"The film opens with Naruto Uzumaki, Sakura Har...",Action


In [13]:
# Look at the distribution of your movies in the dataset
print(top150['Genre'].value_counts())

Science Fiction    76
Drama              47
Comedy             18
Action              9
Name: Genre, dtype: int64


### Part 2: Turn movies into BoW representation (Lecture 7) (5pts)

Turn each movie plot summary (i.e. the 'Text' column in the top150 dataframe) into:

* **Bag-of-Words**

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import warnings
warnings.simplefilter("ignore")

In [76]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(top150.Text)
bow = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())
bow.head()

,000,00am,02,03,04,09,10,100,1000,10__pokemon__a_catchy_toon_crea,...,zoo,zoos,zorgon,zorgons,zubat,zuko,zuul,ángel,über,ōzaru
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* **Bag-of-bigrams**

In [81]:

vectorizer = CountVectorizer(stop_words='english',ngram_range=(2,2))
X = vectorizer.fit_transform(top150.Text)
bo2g = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())
bo2g.head()

,000 000,000 aaron,000 earlier,000 inheritance,000 irs,000 nbsp,000 organize,000 stars,000 words,000 years,...,ángel track,ángel tries,ángel wants,ángel wears,über morlock,über morlocks,ōzaru having,ōzaru return,ōzaru roshi,ōzaru seven
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* **Bag-of-ngrams (for n = 1 and 2)**

In [82]:

vectorizer = CountVectorizer(stop_words='english',ngram_range=(1,2))
X = vectorizer.fit_transform(top150.Text)
bo2g = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())
bo2g.head()

,000,000 000,000 aaron,000 earlier,000 inheritance,000 irs,000 nbsp,000 organize,000 stars,000 words,...,ángel wants,ángel wears,über,über morlock,über morlocks,ōzaru,ōzaru having,ōzaru return,ōzaru roshi,ōzaru seven
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* **TF-IDF values**

In [84]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(top150.Text)
X_train_counts.shape

(150, 11365)

In [87]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tf = tfidf_transformer.fit_transform(X_train_counts)
print(tf)

  (0, 11303)	0.014287749921985657
  (0, 11222)	0.014287749921985657
  (0, 11178)	0.042257345500090665
  (0, 11134)	0.024831369463230914
  (0, 11104)	0.03227592863398996
  (0, 11103)	0.013267527761178665
  (0, 11098)	0.014939973797183127
  (0, 11095)	0.0061155063846792595
  (0, 11048)	0.018291522984151848
  (0, 11030)	0.010599821894287702
  (0, 11026)	0.038471260361307776
  (0, 10893)	0.015911948199795262
  (0, 10756)	0.027569707303357775
  (0, 10729)	0.03504765653685368
  (0, 10724)	0.016801832353584824
  (0, 10698)	0.011112939684303212
  (0, 10687)	0.027569707303357775
  (0, 10647)	0.018742089931023075
  (0, 10579)	0.016487902707806323
  (0, 10538)	0.008215107404036553
  (0, 10500)	0.012057046880805252
  (0, 10396)	0.022824921830578205
  (0, 10391)	0.019781213614718872
  (0, 10356)	0.025470106284000482
  (0, 10343)	0.019781213614718872
  :	:
  (149, 3870)	0.0926091439578152
  (149, 3812)	0.11556153489599158
  (149, 3799)	0.057380101608782236
  (149, 3684)	0.08856773560613052
  (149, 3

### Part 3: Clustering (Lecture 6) (5pts)

Cluster the movies using the k-means algorithm.

**Important Note**: In order to allow you to work on Part 3, before Part 2, the [Movies](https://drive.google.com/drive/folders/1u_cDY-ZIJQKZSdgadzsbI0Q_dj4Jhy41?usp=sharing) folder also contains a comma-separated file, moviesrep.csv, that includes a representation I built for you from the movies plot summary. Each line corresponds to a vector representation (i.e. a set of features) for each movie and you can apply the clustering algorithm on this.

**k-means**

+ Choose the number of clusters you wish to find in the data (n_clusters)
+ Take Bag-of-words and TF-IDF representations from Part 2
+ Run a k-means algorithm
+ Evaluate the quality of the algorithm using inertia_, silhouette_score, adjusted_mutual_info_score, and adjusted_rand_score
    + some of these metrics require the use of ground truth
    + use the genre of each movie as ground truth, i.e. the perfect clustering would be the one that clusters movies based on their genre

In [ ]:
# your code goes here

**Number of cluster**

* Change the value of n\_clusters and plot inertia_, silhouette_score, adjusted_mutual_info_score, and adjusted_rand_score as a function of n_clusters
* Explain what you observe in the plots.
* Do the same for each text representation from Part 2.
* Explain the differences across different representations if there are any

In [ ]:
# your code goes here

*your explanations go here*

**Demonstrate clusters**

* for each representation choose the optimal number of clusters and repeat the k-means algorithm for that number of clusters
* print the top-10 most important words within each cluster
* print the titles of the movies for each cluster
* explain what you observe and whether results make sense

In [ ]:
# your code goes here

*your explanations go here*